# [Day 20 - Jurassic Jigsaw](https://adventofcode.com/2020/day/20)
## Part 1

In [1]:
from math import prod

raw_images = open("inputs/20-input.txt").read().strip().split("\n\n")
images = {int(i[5:9]):i[11:].splitlines() for i in raw_images}

# List of borders of image (read clockwise)
def get_borders(img):
    borders = [
        # Top
        img[0], 
        # Right
        "".join([l[-1] for l in img]),
        # Bottom (reverse column order)
        img[-1][::-1], 
        # Left (reverse row order)
        "".join([l[0] for l in img[::-1]])
    ] 
    # Return borders + reversed borders
    return borders + [b[::-1] for b in borders]


# List of shared borders for border lists a and b
def shared_borders(a:list, b:list):
    return [a_border for b_border in b for a_border in a if a_border == b_border]

# Dict of neighbours for each tile
neighbours = {a :[b for b in images if a != b and shared_borders(get_borders(images[a]), get_borders(images[b]))] for a in images}

# Images with 2 shared borders
corners = [i for i in images if len(neighbours[i]) == 2]
print(corners)
prod(corners)

[1621, 3547, 3389, 1657]


32287787075651

## Part 2

Shit...have to actually **_assemble_** these images

OK, lets pick a top-left corner and go from there...

### Image manipulations

In [2]:
# Rotate tile i 90deg anticlockwise n times
def rotate(img, n):
    arr = np.array([list(row) for row in img])
    arr = np.rot90(arr, n)
    return ["".join(s) for s in arr.tolist()]

# Flip tile, if axis set (0 - rows, 1 - columns)
def flip(img, axis):
    if axis in [0,1]:
        arr = np.array([list(row) for row in img])
        arr = np.flip(arr, axis)
        return ["".join(s) for s in arr.tolist()]
    else:
        return img

# Print clean form of image
def show(img):
    img = "\n".join(img)
    print(img)
    return None

In [3]:
import numpy as np

# Identify neighbour to specified side
def get_neighbour(a, side=["top", "right", "bottom", "left"], idict=images, save=False):
    sides = ["top", "right", "bottom", "left"]

    # Border on specified side
    border = get_borders(idict[a])[sides.index(side)]
    
    # Find corresponding neighbour
    try:
        [neighbour] = [n for n in neighbours[a] if border in shared_borders(get_borders(idict[a]), get_borders(images[n]))]
    except ValueError:
        #print(f"No neighbour on {side} of Tile {a}")
        return None

    # Which of neighbour's borders matches 
    index = get_borders(images[neighbour]).index(border)
    
    # Rotations:
    # +1 in index = 90 deg clockwise
    # index (e.g. 2, bottom) must be 2 apart from side (0, top)
    # So (index - side + 2) rotations anticlockwise required
    n_rotations = (index - sides.index(side) + 2) % 4

    # Flips:
    # If index < 4, matching borders are equal (when read clockwise)
    # To line up, one must be reversed (i.e. read anticlockwise)
    # flip_axis = 0 (rows, top/bottom) / 1 (columns, left/right)
    flip_axis = sides.index(side)-1 % 2 if index < 4 else -1

    # Save transformed neighbour tile directly
    if save:
        idict[neighbour] = flip(rotate(images[neighbour], n_rotations), flip_axis)
    
    return neighbour, n_rotations, flip_axis



### Assembling image by joining tiles

In [4]:
from copy import deepcopy
import numpy as np

# Array to put transformed tiles into
tile_array = np.zeros(shape=(12,12), dtype=int)

# Top-left corner tile
[top_left] = [c for c in corners if get_neighbour(c, "right") and get_neighbour(c, "bottom")]

# Initialise loop and new image dict
i = top_left
trans_images = {}
trans_images[i] = images[i]
next_i, n_rotations, flip_axis = 0,0,-1
for y in range(12):
    # Set start tile for row y
    tile_array[0,y] = i
    
    # Get tile to the right and save to trans_images
    for x in range(1,12):
        next_i, n_rotations, flip_axis = get_neighbour(i, "right", trans_images, save=True)
        tile_array[x,y] = next_i
        i = next_i

    # On completion of row, initialise next row
    if y < 11:
        i = tile_array[0,y]
        next_i, n_rotations, flip_axis = get_neighbour(i, "bottom", trans_images, save=True)
        i = next_i

In [5]:
# Turn image into 2d numpy array
def img_to_arr(i, trim=True):
    if trim:
        return np.array([list(row)[1:-1] for row in trans_images[i][1:-1]])
    else:
        return np.array([list(row) for row in trans_images[i]])

# Stitch trimmed tiles together to a single image
def final_image():
    rows = [np.concatenate([img_to_arr(n) for n in tile_array[:,i]], axis=1) for i in range(12)]
    final = np.concatenate(rows, axis=0)
    return ["".join(s) for s in final.tolist()]

show(final_image())

#...#.#........###.....#####.......#.#.#.#.#...##.#....#.##.#.#.#........#...#.#...#....##....#.
.#............##.....#....#...........##..#..##........##.#.....#.......#.......................
.....#....#..#.........###.....#........#..........###.........#....#..##..............#.....#..
#..##.#..##.#..##.#..###......#.......#...#..#...#..#..#..#..#..#.....#........##..##.#...#...#.
..#..#...###..#.##....###...........#.........#.#...###....##....###..#.....#..........#......#.
#..#.##..#..##.....#...#....###..##.....#.#..........#.##.........#.....#...##.##....#..........
##.#..#..#.......#.#...#.....#..#...#...###..##................#...##...#...#.....#..#......#.#.
.....#..#.......#...###.......##.........#.#....#....#..#.#.#..#.......##...#...................
.....#.#....#.......#........##....#.....#....#...##..#####.#..#.###.##...........#...........#.
.......#...................#....#........##.####...###....##..#.##...####.#...#.#.##...........#
........#.......#..........#.#

### Find and count monsters

In [6]:
monster = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """.splitlines()

# Coordinate of hashes relative to top left of monster image
hash_indices = [[x,y] for y in range(len(monster)) for x in range(len(monster[0])) if monster[y][x] == '#']

monster_count = 0

# Try all orientations of final image
for img in [flip(rotate(final_image(),i),j) for i in range(4) for j in [-1,1]]:
    # Try all starting coordinates [x,y]
    for y in range(len(img)-len(monster)):
        for x in range(len(img[0])-len(monster[0])):
            # If all hash_indices relative to [x,y] match - MONSTER!
            if all([img[y+j][x+i] == '#' for i,j in hash_indices]):
                monster_count += 1

# Subtract hashes in monsters from total hashes in image
all_hash_count = "".join([l for l in final_image()]).count("#")
monster_hash_count = monster_count * "".join([l for l in monster]).count("#")
print(all_hash_count - monster_hash_count)

1939
